In [1]:
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from skimage.io import imread, imsave
from skimage.filters import median, threshold_otsu, sobel, gaussian
from skimage.morphology import disk
from skimage.measure import label, regionprops
from skimage.color import label2rgb, gray2rgb
from skimage.segmentation import mark_boundaries, clear_border
from skimage.draw import circle_perimeter
import pandas as pd
from scipy import ndimage
from skimage.restoration import rolling_ball as ball
from skimage.exposure import equalize_adapthist as histEq
from glob import glob
from time import time
import napari   #this code imports all of the required packages 

In [ ]:
def first_step(filePath):
    imgOrig = imread(filePath)    #reads the image in from the provided file path
    redImg   = imgOrig[:, :, 0]
    greenImg = imgOrig[:, :, 1]
    blueImg  = imgOrig[:, :, 2]   #this was me exploring reading in a colour image, and splitting it into the r,g,b channels
    
    imgSize = imgOrig.shape       #finds the image shape
    imgType = type(imgOrig)       #finds the image type
    pixValType = imgOrig.dtype    #finds pixel type (i.e. Uint8 or float64 etc)
    pixValMin = np.min(imgOrig)   #finds max pixel value
    pixValMax = np.max(imgOrig)   #finds min pixel value

    
    displayWidth  = 7   
    displayHeight = (imgSize[0]/imgSize[1])*displayWidth   #maintains aspect ratio
    plt.figure(figsize=(displayHeight, displayWidth))
    plt.imshow(imgOrig,cmap=plt.cm.gray)   # need to provide color map for plt
    plt.show()   #this displays the image read 
     
    return imgOrig, imgSize, imgType, pixValType, pixValMin, pixValMax, redImg, greenImg, blueImg

    

In [ ]:

from skimage.segmentation import watershed
from scipy import ndimage as ndi
from skimage import morphology
from skimage.feature import peak_local_max   #this imports all the required packages etc 



def watershedFunction(image, x, y):
    
    '''
    Input: Binary image: image, min_distance: x, 
    scaling factor of threshold_abs: y

    Function: Takes a binary image, carries out a distance transform 
    and a peak search. Then creates an array of zeros but with 
    identified peaks given a number (instead of zero). Then carries
    out a watershed from the peaks and distance transform, which is 
    then passed into the skimage.measure label function that creates 
    and assigns labels to the grouped regions (the particles in this 
    instance).

    Output: distance transform image, watershed threshold image, 
    the labels, and number of labels

    '''

#this bit of code inverts the binary image, as the distance map 
#would be acting on the background, which is more adjacent to a 
# Voronoi map as opposed to using it to find local maxima 
    
    if np.mean(image) > 0.5:    
        image = ~image         

#creates the distance transform as seen below in the middle image 
    distance = ndi.distance_transform_edt(image)    

    coords = peak_local_max(distance, 
                            min_distance = x,
                            threshold_ab3s = 0.2 * distance.max(), 
                            labels = image)  #finds the local maxima in 
                            #the distance map (above line)
    markers = np.zeros(distance.shape, dtype =int)
    for i, coord in enumerate(coords, start = 1):
        markers[coord[0], coord[1]] = i             #

#applies a watershed on the negative of the distance map (because we are     
#looking for peaks, not basins which is how the watershed works).
    
    segmentation = watershed(-distance, markers, mask = image) 
                                                               
#labels the now watershed particles  
    labels, num = label(segmentation, background = 0, return_num = True)  

    return distance, segmentation, labels, num 
#runs the function above 
distance, segmentation, labels, num = watershedFunction(imgBin, 3, 0.5)   

print(r'The total number of watershed regions identified: ', num)     

#plots the figures
plt.figure(figsize=(18, 8), dpi = 500)

plt.subplot(1, 3, 1)
plt.imshow(imgOrig, cmap='gray')
plt.title('Original Image')

plt.subplot(1, 3, 2)
plt.imshow(distance, cmap='magma')
plt.title('Distance Transform')

plt.subplot(1, 3, 3)
plt.imshow(segmentation, cmap='nipy_spectral')
plt.title('Watershed Segmentation')

plt.show()


In [ ]:
def plotHistogram(pixValMin, pixValMax, imgOrig):
    binsSeq           = np.arange(pixValMin-0.5, pixValMax+1.5, 1.0) #creates a sequence encapsulating the entirety of the pixel range
    histVal, binEdges = np.histogram(imgOrig.flatten(), bins=binsSeq)
    binCentres        = 0.5*(binEdges[:-1]+binEdges[1:])
    print("First few bin edges  : {}".format(binEdges[:5]))
    print("Final few bin edges  : {}".format(binEdges[-5:]))
    print("First few bin centres: {}".format(binCentres[:5]))
    print("Final few bin centres: {}".format(binCentres[-5:]))  #prints some checks with bin edges and centres

    plt.figure(figsize=(4,2))
    plt.bar(binCentres, histVal)
    plt.xlabel("pixel value")
    plt.ylabel("count")  #plots the histogram
    plt.show()
    return 

plotHistogram(pixValMin, pixValMax, imgOrig)



In [ ]:
def fourierFunction(imgOrig):
    cropImg = imgOrig  
    fftImg = np.fft.fft2(cropImg)           #fast fourier transform
    shifted_FFT = np.fft.fftshift(fftImg)   #shifts the 4 corners into the centre 
    shiftedFFT = np.abs(shifted_FFT)**2     #finds the absolute square magnutide 
    power2dlog = np.log1p(shiftedFFT)       #takes the log (in order to plot it in log space)
    plt.figure(figsize=(9, 6))
    plt.imshow(np.log1p(shiftedFFT), cmap = plt.cm.gray)     #plots the above 2d power spectrum

    max_truple = np.unravel_index(shiftedFFT.argmax(), shiftedFFT.shape)
    print(max_truple)
    return power2dlog, shifted_FFT

power2dlog, shifted_FFT = fourierFunction(redCrop)

def powerSpectrum1D(image):
    
    x0 = 800
    y0 = 800   #should change this to not be hardcoded in, i.e. pass in the centre of the image 
    x,y = np.meshgrid(np.arange(image.shape[1]),np.arange(image.shape[0]))
    R = np.sqrt((x-x0)**2+(y-y0)**2)
    #print(R)

    f = lambda r : image[(R >= r-0.1) & (R < r+0.1)].mean()
    r  = np.linspace(1,800,num=800)       #this takes the radial average of the 2d power spectrum found above 
    mean = np.vectorize(f)(r)

    fig,ax=plt.subplots()
    ax.plot(r,mean)
    plt.xlim(0,100)
    plt.show()
    
    return 
    
powerSpectrum1D(power2dlog)

In [ ]:
def highPassFilter(size, center, radius):
    arr = np.ones((size, size))  #creates an array of 1s the same size as the image

    y, x = np.ogrid[:size, :size]  #create meshgrid 

    dist_sq = (x - center)**2 + (y - center)**2   #finds the square distance from centre

    arr[dist_sq < radius**2] = 0  #any pixel within that distance is set to value 0
    
    plt.imshow(arr, cmap='gray')   #plots the high pass array
    plt.show()

    return arr

highMask = highPassFilter(1600, 800, 20)    #this is bad because of the edge effects etc (sinc function same as finite mirror astronomy)

In [ ]:
def butterworth(size, center, radius):
    arr = np.ones((size, size))

    y, x = np.ogrid[:size, :size]  #create meshgrid 
 
    dist_sq = (x - center)**2 + (y - center)**2
    #print(dist_sq[,100])

    for i in range(0,size):
        for j in range(0,size):
            
            arr[i,j] =  bhpf(np.sqrt(dist_sq[i,j]))   #this is very similar to above, the only difference is that the butterworth  function is used over the whole image
        
    plt.imshow(arr, cmap='gray')
    plt.show()

    return arr

butterworth1 = butterworth(200, 100, kc)

In [ ]:
def RollingBall(image, rad):
    background = ball(image, radius = rad)   #finds the background (which is usually a gradient from uneven illumination)
    corrected_image = image - background  #subtracts the uneven background from the image 
    return corrected_image

testImgCUfiltBGC = RollingBall(testImgCU, 25)

def HistEqualise(image):
    histshift = histEq(image)  #this function is a bit redundant and could just be a line of code?
    
    return histshift

imgRest = HistEqualise(testImgCUfiltBGC)

imgRest2 = 255*imgRest

print(imgRest2.max())
print(imgRest2.min())